In [42]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [43]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [44]:
user_item_matrix = df.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix = user_item_matrix.fillna(0)

In [45]:
user_item_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
item_similarity = cosine_similarity(user_item_matrix.T)

In [47]:
import numpy as np
userId = 1 # replace with the id of the user you want to predict ratings for
user_ratings = user_item_matrix.loc[userId].values
user_predicted_ratings = np.dot(item_similarity, user_ratings) / np.sum(item_similarity, axis=1)


In [48]:
df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [49]:
# Assuming 'movie_ids' contains the list of movie IDs
movie_ids = user_item_matrix.columns

# Create a DataFrame to combine movie IDs with their predicted ratings
predicted_ratings_df = pd.DataFrame({
    'movieId': movie_ids,
    'predicted_rating': user_predicted_ratings
})

# Merge with movie information (if available) for better readability
predicted_ratings_with_info = predicted_ratings_df.merge(df_movies, on='movieId', how='left')

# Display the top N predicted ratings
top_n_predictions = predicted_ratings_with_info.nlargest(5, 'predicted_rating')
print(top_n_predictions)


      movieId  predicted_rating  \
1125     1473          4.439465   
1325     1793          2.269539   
2181     2899          1.754489   
666       876          1.356194   
4218     6145          1.323607   

                                               title  \
1125                                 Best Men (1997)   
1325                     Welcome to Woop-Woop (1997)   
2181                       Gulliver's Travels (1939)   
666   Supercop 2 (Project S) (Chao ji ji hua) (1993)   
4218                                    Venom (1982)   

                            genres  
1125     Action|Comedy|Crime|Drama  
1325                        Comedy  
2181  Adventure|Animation|Children  
666   Action|Comedy|Crime|Thriller  
4218               Horror|Thriller  


In [50]:
def get_actual_rating(user_id, movie_id):
    user_movie_rating = df[(df['userId'] == user_id) & (df['movieId'] == movie_id)]['rating'].values
    if len(user_movie_rating) > 0:
        return user_movie_rating[0]
    else:
        return None

4.0

In [57]:
from sklearn.metrics import mean_absolute_error
userId=1
movie_ids = user_item_matrix.columns[1]
# Assuming 'actual_ratings' is a list of actual ratings corresponding to the predicted ratings
actual_ratings = [get_actual_rating(userId, movie_id) for movie_id in movie_ids]  # Replace with your actual ratings retrieval function

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(actual_ratings, user_predicted_ratings)
print(f'MAE: {mae}')


TypeError: 'numpy.int64' object is not iterable